In [ ]:
from urllib.request import urlretrieve
import zipfile, os
import pandas as pd
import numpy as np
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_train.csv" "/content"
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_ICM_title_abstract.csv" "/content"
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_target_users_test.csv" "/content"
!unzip '/content/drive/My Drive/Colab Notebooks/Recommender System/RecSys_Course_AT_PoliMi-master.zip'
!cp  "/content/RecSys_Course_AT_PoliMi-master/Base/Evaluation/Evaluator.py" "/content"

sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Data_manager/split_functions')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master')



URM_all_dataframe = pd.read_csv("/content/data_train.csv", 
                                sep=",", 
                                names=["user_id", "item_id", "interaction"],
                                header=0,
                                dtype ={"row": np.int32,
                                "col": np.int32,
                                "data": np.int32})

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]


ICM_dataframe = pd.read_csv("/content/data_ICM_title_abstract.csv", 
                            sep=",", 
                            names=["item_id", "feature_id", "importance"],
                            header=0,
                            dtype={"row": np.int32,
                              "col": np.int32,
                              "data": np.float})


ICM_dataframe.columns = ["item_id", "feature_id", "importance"]

URM_all_dataframe_old = URM_all_dataframe
ICM_dataframe_old = ICM_dataframe

In [ ]:
item_original_ID_to_index_dict = {}
map_new_into_old = {}

for item_id in URM_all_dataframe["item_id"].unique():
    item_original_ID_to_index_dict[item_id] = len(item_original_ID_to_index_dict)
    map_new_into_old[item_original_ID_to_index_dict[item_id]] = item_id

print("Unique item_id in the URM are {}".format(len(item_original_ID_to_index_dict)))
    
for item_id in ICM_dataframe["item_id"].unique():
    if item_id not in item_original_ID_to_index_dict:
        item_original_ID_to_index_dict[item_id] = len(item_original_ID_to_index_dict)
        
print("Unique item_id in the URM and ICM are {}".format(len(item_original_ID_to_index_dict)))




feature_original_ID_to_index_dict = {}

for feature_id in ICM_dataframe["feature_id"].unique():
    feature_original_ID_to_index_dict[feature_id] = len(feature_original_ID_to_index_dict)

print("Unique feature_id in the URM are {}".format(len(feature_original_ID_to_index_dict)))




URM_all_dataframe["item_id"] = [item_original_ID_to_index_dict[item_original] for item_original in 
                                      URM_all_dataframe["item_id"].values]



                                      
ICM_dataframe["item_id"] = [item_original_ID_to_index_dict[item_original] for item_original in 
                                      ICM_dataframe["item_id"].values]

ICM_dataframe["feature_id"] = [feature_original_ID_to_index_dict[feature_original] for feature_original in 
                                      ICM_dataframe["feature_id"].values]

Unique item_id in the URM are 24896
Unique item_id in the URM and ICM are 25975
Unique feature_id in the URM are 19998


In [ ]:
import scipy.sparse as sps
import numpy as np

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(item_original_ID_to_index_dict)
n_features = len(feature_original_ID_to_index_dict)

URM_all = sps.csr_matrix((URM_all_dataframe["interaction"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))


ICM_all_importance = sps.csr_matrix((ICM_dataframe["importance"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all = sps.csr_matrix((np.ones(len(ICM_dataframe["item_id"].values)), 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

**TRAIN/TEST SPLIT**

In [ ]:
import sys
sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Data_manager/split_functions')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Utils')


from Evaluator import EvaluatorHoldout
from split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)
URM_train_validation = URM_train + URM_validation

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

recommender2 = RP3betaRecommender(URM_train_validation)


#recommender2.fit(topK=92, alpha=0.4669466039587683, beta=0.004657209415055234, normalize_similarity=False)  #should be best
#recommender2.fit(topK=750, alpha=0.42462171653398106, beta=0.01, normalize_similarity=False)
#recommender2.fit(topK=514, alpha=0.44944931756992157, beta=3.860008962677021e-05, normalize_similarity=False)
#recommender2.fit(topK=300, alpha=0.3858658184080451, beta=0.01, normalize_similarity=False)  #should be second
#recommender2.fit(topK=500, alpha=0.4766038362068743, beta=0.0, normalize_similarity=False) 
#recommender2.fit(topK=1446, alpha=0.6464084702863461, beta=0.005818963680425116, normalize_similarity=False)
#recommender2.fit(topK=70, alpha=0.3522657797468572, beta=0.0062960436321015684, normalize_similarity=False)

result_dict, _ = evaluator_test.evaluateRecommender(recommender2)
result_dict

RP3betaRecommender: URM Detected 40 (0.50 %) cold users.
RP3betaRecommender: URM Detected 2003 (7.71 %) cold items.
EvaluatorHoldout: Processed 4957 ( 100.00% ) in 2.66 sec. Users per second: 1862


{10: {'ARHR': 0.13221786730965682,
  'AVERAGE_POPULARITY': 0.11336295825077237,
  'COVERAGE_ITEM': 0.4202502406159769,
  'COVERAGE_ITEM_CORRECT': 0.03437921077959576,
  'COVERAGE_USER': 0.6237573927268152,
  'COVERAGE_USER_CORRECT': 0.15741789354473387,
  'DIVERSITY_GINI': 0.13524886863490446,
  'DIVERSITY_HERFINDAHL': 0.9982916116731794,
  'DIVERSITY_MEAN_INTER_LIST': 0.9831144452460653,
  'F1': 0.05122337203754197,
  'HIT_RATE': 0.32297760742384507,
  'MAP': 0.060264442096969656,
  'MRR': 0.11688681390113714,
  'NDCG': 0.08853739598720502,
  'NOVELTY': 0.004832341282358952,
  'PRECISION': 0.03229776074238388,
  'PRECISION_RECALL_MIN_DEN': 0.126470423419183,
  'RECALL': 0.12371984182616745,
  'ROC_AUC': 0.16195088875439934,
  'SHANNON_ENTROPY': 11.35322967345475}}

In [ ]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

#shr 45 k 120  = 0.054
#shr 180 k 534 = 0.0535


recommender = ItemKNNCFRecommender(URM_train_validation)
recommender.fit(shrink=649, topK=643, similarity="asymmetric")


result_dict, _ = evaluator_test.evaluateRecommender(recommender)
result_dict

ItemKNNCFRecommender: URM Detected 28 (0.35 %) cold users.
ItemKNNCFRecommender: URM Detected 1951 (7.51 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 1740.36 column/sec, elapsed time 0.25 min
EvaluatorHoldout: Processed 4995 ( 100.00% ) in 3.31 sec. Users per second: 1511


{10: {'ARHR': 0.11005783561339122,
  'AVERAGE_POPULARITY': 0.14484434590353648,
  'COVERAGE_ITEM': 0.36408084696823867,
  'COVERAGE_ITEM_CORRECT': 0.025332050048123195,
  'COVERAGE_USER': 0.6285390713476784,
  'COVERAGE_USER_CORRECT': 0.1291053227633069,
  'DIVERSITY_GINI': 0.09707859229725568,
  'DIVERSITY_HERFINDAHL': 0.9970221095970846,
  'DIVERSITY_MEAN_INTER_LIST': 0.9704153733228623,
  'F1': 0.04175565431569963,
  'HIT_RATE': 0.26166166166166166,
  'MAP': 0.05017511455143724,
  'MRR': 0.0986116275005163,
  'NDCG': 0.07369106956128334,
  'NOVELTY': 0.004702938285331611,
  'PRECISION': 0.026166166166165766,
  'PRECISION_RECALL_MIN_DEN': 0.10557271557271539,
  'RECALL': 0.10330137567613147,
  'ROC_AUC': 0.13378889206666997,
  'SHANNON_ENTROPY': 10.604659104247068}}

**HYPERPARAMETER TUNING**

In [ ]:
%%time
#jaccard
#shr = 45
#k= 120
import copy
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

similarities = ['jaccard']
values = [40,50,70, 90, 100, 120,130 ,150,200,300]
MAP_per_shrink = {}
MAP_per_sim = {}

for sim in similarities:
  for val in values:  
      print("EVALUATING shr: " + str(val))
      recommender = ItemKNNCFRecommender(URM_train)
      recommender.fit(shrink=45, topK=val, similarity=sim)
      
      result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
      
      MAP_per_shrink[val] = (result_dict[10]["MAP"])
  MAP_per_sim[sim] = copy.deepcopy(MAP_per_shrink)

**HYPERPARAMETER TUNING WITH BAYESIAN SEARCH**

In [ ]:
pip install h5py scikit-optimize

     |████████████████████████████████| 102kB 4.0MB/s 


In [ ]:
%%time
import skopt
from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
import os
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender


#hyperparameters_range_dictionary = {}
#hyperparameters_range_dictionary["topK"] = Integer(10, 1000)
#hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
#hyperparameters_range_dictionary["similarity"] = Categorical(["asymmetric"])
#hyperparameters_range_dictionary["normalize"] = Categorical([True, False])

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["topK"] = Integer(230, 290)
hyperparameters_range_dictionary["alpha"] = Real(low = 0.3, high = 0.7, prior = 'uniform')
hyperparameters_range_dictionary["beta"] = Real(low = 0, high = 0.01, prior = 'uniform')
hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])

n_cases = 1000
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"



recommender_class = RP3betaRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)



  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)




recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)




output_folder_path = "/content/result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    




parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
RP3betaRecommender: URM Detected 176 (2.21 %) cold users.
RP3betaRecommender: URM Detected 3639 (14.01 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 280, 'alpha': 0.4509817455854424, 'beta': 0.0062302825346026835, 'normalize_similarity': True}
EvaluatorHoldout: Processed 5268 ( 100.00% ) in 3.48 sec. Users per second: 1515
SearchBayesianSkopt: New best config found. Config 0: {'topK': 280, 'alpha': 0.4509817455854424, 'beta': 0.0062302825346026835, 'normalize_similarity': True} - results: ROC_AUC: 0.1236917, PRECISION: 0.0258922, PRECISION_RECALL_MIN_DEN: 0.0888598, RECALL: 0.0859504, MAP: 0.0378368, MRR: 0.0877285, NDCG: 0.0600991, F1: 0.0397960, HIT_RATE: 0.2589218, ARHR: 0.0990685, NOVELTY: 0.0051411, AVERAGE_POPULARITY: 0.0631353, DIVERSITY_MEAN_INTER_LIST: 0.9932442, DIVERSITY_HERFINDAHL: 0.9993056, COVERAGE_ITEM: 0.5247353, COVERAGE_ITEM_CORRECT: 0.0330318, COVERAGE_USER: 0.6628917, COVERAGE_USER_CORRECT

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5268 ( 100.00% ) in 2.75 sec. Users per second: 1917
SearchBayesianSkopt: Config 304 is suboptimal. Config: {'topK': 230, 'alpha': 0.3, 'beta': 0.0, 'normalize_similarity': False} - results: ROC_AUC: 0.1383797, PRECISION: 0.0278284, PRECISION_RECALL_MIN_DEN: 0.1025268, RECALL: 0.0996270, MAP: 0.0456597, MRR: 0.0995517, NDCG: 0.0701639, F1: 0.0435048, HIT_RATE: 0.2782840, ARHR: 0.1106650, NOVELTY: 0.0047556, AVERAGE_POPULARITY: 0.1189912, DIVERSITY_MEAN_INTER_LIST: 0.9800979, DIVERSITY_HERFINDAHL: 0.9979912, COVERAGE_ITEM: 0.3768239, COVERAGE_ITEM_CORRECT: 0.0282964, COVERAGE_USER: 0.6628917, COVERAGE_USER_CORRECT: 0.1443312, DIVERSITY_GINI: 0.1096796, SHANNON_ENTROPY: 11.0488826, 

Iteration No: 305 ended. Search finished for the next optimal point.
Time taken: 27.0000
Function value obtained: -0.0457
Current minimum: -0.0467
Iteration No: 306 started. Searching for the next optimal point.
RP3betaRecommender: URM Detected 176 (2.21 %) cold users.
RP3betaReco

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5268 ( 100.00% ) in 2.76 sec. Users per second: 1911
SearchBayesianSkopt: Config 307 is suboptimal. Config: {'topK': 230, 'alpha': 0.3, 'beta': 0.0, 'normalize_similarity': False} - results: ROC_AUC: 0.1383797, PRECISION: 0.0278284, PRECISION_RECALL_MIN_DEN: 0.1025268, RECALL: 0.0996270, MAP: 0.0456597, MRR: 0.0995517, NDCG: 0.0701639, F1: 0.0435048, HIT_RATE: 0.2782840, ARHR: 0.1106650, NOVELTY: 0.0047556, AVERAGE_POPULARITY: 0.1189912, DIVERSITY_MEAN_INTER_LIST: 0.9800979, DIVERSITY_HERFINDAHL: 0.9979912, COVERAGE_ITEM: 0.3768239, COVERAGE_ITEM_CORRECT: 0.0282964, COVERAGE_USER: 0.6628917, COVERAGE_USER_CORRECT: 0.1443312, DIVERSITY_GINI: 0.1096796, SHANNON_ENTROPY: 11.0488826, 

Iteration No: 308 ended. Search finished for the next optimal point.
Time taken: 27.1540
Function value obtained: -0.0457
Current minimum: -0.0467
Iteration No: 309 started. Searching for the next optimal point.
RP3betaRecommender: URM Detected 176 (2.21 %) cold users.
RP3betaReco

...

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5268 ( 100.00% ) in 2.76 sec. Users per second: 1912
SearchBayesianSkopt: Config 540 is suboptimal. Config: {'topK': 230, 'alpha': 0.3, 'beta': 0.0, 'normalize_similarity': False} - results: ROC_AUC: 0.1383797, PRECISION: 0.0278284, PRECISION_RECALL_MIN_DEN: 0.1025268, RECALL: 0.0996270, MAP: 0.0456597, MRR: 0.0995517, NDCG: 0.0701639, F1: 0.0435048, HIT_RATE: 0.2782840, ARHR: 0.1106650, NOVELTY: 0.0047556, AVERAGE_POPULARITY: 0.1189912, DIVERSITY_MEAN_INTER_LIST: 0.9800979, DIVERSITY_HERFINDAHL: 0.9979912, COVERAGE_ITEM: 0.3768239, COVERAGE_ITEM_CORRECT: 0.0282964, COVERAGE_USER: 0.6628917, COVERAGE_USER_CORRECT: 0.1443312, DIVERSITY_GINI: 0.1096796, SHANNON_ENTROPY: 11.0488826, 

Iteration No: 541 ended. Search finished for the next optimal point.
Time taken: 45.1707
Function value obtained: -0.0457
Current minimum: -0.0467
Iteration No: 542 started. Searching for the next optimal point.
RP3betaRecommender: URM Detected 176 (2.21 %) cold users.
RP3betaReco

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5268 ( 100.00% ) in 2.76 sec. Users per second: 1908
SearchBayesianSkopt: Config 542 is suboptimal. Config: {'topK': 230, 'alpha': 0.3, 'beta': 0.01, 'normalize_similarity': False} - results: ROC_AUC: 0.1385814, PRECISION: 0.0282270, PRECISION_RECALL_MIN_DEN: 0.1025092, RECALL: 0.0995400, MAP: 0.0463759, MRR: 0.1005252, NDCG: 0.0708491, F1: 0.0439819, HIT_RATE: 0.2822703, ARHR: 0.1121202, NOVELTY: 0.0048018, AVERAGE_POPULARITY: 0.1155832, DIVERSITY_MEAN_INTER_LIST: 0.9809473, DIVERSITY_HERFINDAHL: 0.9980761, COVERAGE_ITEM: 0.4141290, COVERAGE_ITEM_CORRECT: 0.0294514, COVERAGE_USER: 0.6628917, COVERAGE_USER_CORRECT: 0.1453379, DIVERSITY_GINI: 0.1258926, SHANNON_ENTROPY: 11.1941800, 

Iteration No: 543 ended. Search finished for the next optimal point.
Time taken: 45.4170
Function value obtained: -0.0464
Current minimum: -0.0467
Iteration No: 544 started. Searching for the next optimal point.
RP3betaRecommender: URM Detected 176 (2.21 %) cold users.
RP3betaRec

KeyboardInterrupt: ignored

**HYPER PARAMETER TUNING ALL RECOMMENDER**

In [ ]:
from Base.NonPersonalizedRecommender import TopPop, Random
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender

import traceback

import os, multiprocessing
from functools import partial



from Data_manager.Movielens.Movielens10MReader import Movielens10MReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from ParameterTuning.run_parameter_search import runParameterSearch_Collaborative


def read_data_split_and_search():
    """
    This function provides a simple example on how to tune parameters of a given algorithm
    The BayesianSearch object will save:
        - A .txt file with all the cases explored and the recommendation quality
        - A _best_model file which contains the trained model and can be loaded with recommender.load_model()
        - A _best_parameter file which contains a dictionary with all the fit parameters, it can be passed to recommender.fit(**_best_parameter)
        - A _best_result_validation file which contains a dictionary with the results of the best solution on the validation
        - A _best_result_test file which contains a dictionary with the results, on the test set, of the best solution chosen using the validation set
    """



   # dataReader = Movielens10MReader()
    #dataset = dataReader.load_data()

    #URM_train, URM_test = split_train_in_two_percentage_global_sample(dataset.get_URM_all(), train_percentage = 0.80)
    #URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

    output_folder_path = "result_experiments_all_alg/"


    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)







    collaborative_algorithm_list = [
        Random,
        TopPop,
        P3alphaRecommender,
        RP3betaRecommender,
        ItemKNNCFRecommender,
        UserKNNCFRecommender,
        MatrixFactorization_BPR_Cython,
        MatrixFactorization_FunkSVD_Cython,
        PureSVDRecommender,
        SLIM_BPR_Cython,
        SLIMElasticNetRecommender
    ]




    #from Base.Evaluation.Evaluator import EvaluatorHoldout

    #evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[5])
   # evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5, 10])


    runParameterSearch_Collaborative_partial = partial(runParameterSearch_Collaborative,
                                                       URM_train = URM_train,
                                                       metric_to_optimize = "MAP",
                                                       n_cases = 500,
                                                       evaluator_validation_earlystopping = evaluator_validation,
                                                       evaluator_validation = evaluator_validation,
                                                       evaluator_test = evaluator_test,
                                                       output_folder_path = output_folder_path,
                                                       similarity_type_list = ["asymmetric", "jaccard"],
                                                       parallelizeKNN = False)





    pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
    pool.map(runParameterSearch_Collaborative_partial, collaborative_algorithm_list)

    #
    #
    # for recommender_class in collaborative_algorithm_list:
    #
    #     try:
    #
    #         runParameterSearch_Collaborative_partial(recommender_class)
    #
    #     except Exception as e:
    #
    #         print("On recommender {} Exception {}".format(recommender_class, str(e)))
    #         traceback.print_exc()
    #







if __name__ == '__main__':


    read_data_split_and_search()

**SUBMISSION**

In [ ]:
#3 min 44 sec
users = pd.read_csv("/content/data_target_users_test.csv", 
                       #sep=",", 
                       names=["user_id"],
                       header=0,
                       dtype={"user_id": np.int32,
                              })
utenti = users.user_id.unique()
print(utenti)
len(utenti)





def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix, recommender: object):
  #  users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id"]].drop_duplicates()
   # items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
    
   # mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    
    
    recommendation_length = 10
    submission = []
    for user_id in users_to_recommend:
        #mapped_user_id = row.mapped_user_id
        
        recommendations = recommender.recommend(user_id,
                                                recommendation_length,
                                                )
        
        submission.append((user_id, [map_new_into_old[item_id] for item_id in recommendations]))
        
    return submission


def write_submission(submissions):
    with open("./submission1.csv", "w") as f:
        f.write(f"{'user_id'},{'item_list'}\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

[   0    1    2 ... 7944 7945 7946]


In [ ]:
#Create Submission

submission = prepare_submission(URM_all_dataframe, utenti, URM_train + URM_validation + URM_test, recommender2)
submission

#25974


In [ ]:
            
write_submission(submission)